In [1]:
from categories import print_tests, all_tests
from rdkit import Chem

[12:08:37] SMARTS Parse Error: syntax error while parsing: [#7](=[#8])$[[#1],[#6X4]]{1..;xm}
[12:08:37] SMARTS Parse Error: Failed parsing SMARTS '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}' for input: '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}'
[12:08:37] SMARTS Parse Error: syntax error while parsing: $[[#1],[#6X4]]{1..;xm}[#7+][#8-]
[12:08:37] SMARTS Parse Error: Failed parsing SMARTS '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]' for input: '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]'
[12:08:37] SMARTS Parse Error: syntax error while parsing: c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}
[12:08:37] SMARTS Parse Error: Failed parsing SMARTS 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}' for input: 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}'
[12:08:37] SMARTS Parse Error: syntax error while parsing: $[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]
[12:08:37] SMARTS Parse Error: Failed parsing SMARTS '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]' for input: '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]'
[12:08:37] SMARTS Parse Error: syntax error while parsing: [#8]{>-

In [2]:
all_tests['Acid Chlorides'].print_tree(None)

('32', 'LogicalQuery', 'And', 'does not process')
	('29', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')
	('26', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', 'does not process')
	('24', 'b:StructureQuery', '[#6,#16](=[#8])(Cl)[#6X4,c]', 'does not process')


In [3]:
# Define a test chemical as chemicals are stored in the GenRA database
test_chem = {'dsstox_sid': 'DTXSID7020009',
 'smiles': 'CC#N',
 'logp': -0.33971,
 'ws': 12.6417,
 'mol_weight': 41.053,
 'mol': Chem.MolFromSmiles('CC#N')}

In [4]:
from categories import queryAll

In [5]:
print_tests

{'Acid Chlorides': ['32', 'LogicalQuery'],
 'Acrylamides': ['51', 'LogicalQuery'],
 'Acrylates/Methacrylates (Acute toxicity)': ['70', 'LogicalQuery'],
 'Aldehydes (Acute toxicity)': ['89', 'LogicalQuery'],
 'Aliphatic Amines': ['121', 'LogicalQuery'],
 'Aluminum Compounds': ['140', 'LogicalQuery'],
 'Anilines (Acute toxicity)': ['159', 'LogicalQuery'],
 'Azides (Acute toxicity)': ['184', 'LogicalQuery'],
 'Benzotriazoles (Acute toxicity)': ['203', 'LogicalQuery'],
 'Benzotriazole-hindered phenols': ['214', 'b:StructureQuery'],
 'Boron Compounds': ['230', 'LogicalQuery'],
 'Cationic (quaternary ammonium) surfactants': ['241', 'b:StructureQuery'],
 'Cobalt': ['263', 'LogicalQuery'],
 'Diazoniums (Acute toxicity)': ['282', 'LogicalQuery'],
 'Epoxides': ['298', 'LogicalQuery'],
 'Esters (Acute toxicity)': ['317', 'LogicalQuery'],
 'Hydrazines and Related Compounds': ['336', 'LogicalQuery'],
 'Hindered Amines': ['350', 'b:StructureQuery'],
 'Imides (Acute toxicity)': ['369', 'LogicalQuery'

In [14]:
df

,chemicals,Acid Chlorides,Acrylamides,Acrylates/Methacrylates (Acute toxicity),Aldehydes (Acute toxicity),Aliphatic Amines,Aluminum Compounds,Anilines (Acute toxicity),Azides (Acute toxicity),Benzotriazoles (Acute toxicity),...,Organotins (Chronic toxicity),Phenols (Chronic toxicity),Phosphinate Esters (Chronic toxicity),Polynitroaromatics (Chronic toxicity),Substituted Triazines (Chronic toxicity),Thiols (Chronic toxicity),Vinyl Esters (Chronic toxicity),Diazoniums (Chronic toxicity),Ethylene Glycol Ethers,Benzotriazoles
DTXSID3060164,DTXSID3060164,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DTXSID3060164,DTXSID3060164,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DTXSID7060837,DTXSID7060837,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DTXSID9025879,DTXSID9025879,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DTXSID2026282,DTXSID2026282,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
DTXSID4052188,DTXSID4052188,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
df = queryAll(new_test_chems[0])
for category in all_tests.keys():
     for chem in df.index:
          if df.loc[chem,category] == True:
               print(category)

Epoxides are still a problem
Persistent, Bioaccumulative and Toxic (PBT) Chemicals are still a problem
Neutral Organics


In [6]:
new_test_chems = [{'dsstox_sid': 'DTXSID3060164',
  'smiles': 'C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1',
  'logp': 5.76,
  'ws': 4.07380277804113e-07,
  'mol_weight': 244.125200512,
  'mol': Chem.MolFromSmiles('C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1')},
 {'dsstox_sid': 'DTXSID7060837',
  'smiles': 'ICCCI',
  'logp': 3.02,
  'ws': 0.0007413102413009177,
  'mol_weight': 295.855896192,
  'mol': Chem.MolFromSmiles('ICCCI')},
 {'dsstox_sid': 'DTXSID9025879',
  'smiles': 'OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1',
  'logp': 1.99,
  'ws': 0.009120108393559097,
  'mol_weight': 218.0579088,
  'mol': Chem.MolFromSmiles('OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1')},
 {'dsstox_sid': 'DTXSID2026282',
  'smiles': 'O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=2C4=O)NC(=O)C2C=CC=CC=2)C2=C1C(=O)C1C=CC=CC=1C2=O)C1C=CC=CC=1',
  'logp': 3.11,
  'ws': 2.454708915685029e-08,
  'mol_weight': 667.174335520001,
  'mol': Chem.MolFromSmiles('O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=2C4=O)NC(=O)C2C=CC=CC=2)C2=C1C(=O)C1C=CC=CC=1C2=O)C1C=CC=CC=1')},
 {'dsstox_sid': 'DTXSID4052188',
  'smiles': 'CC1(C)COC(C)(OC1)C1C=CC=CC=1',
  'logp': 2.72,
  'ws': 0.001148153621496883,
  'mol_weight': 206.130679816,
  'mol': Chem.MolFromSmiles('CC1(C)COC(C)(OC1)C1C=CC=CC=1')}]

In [7]:

epoxide=Chem.MolFromSmarts('c1oc1')
aziridine=Chem.MolFromSmarts('c1cn1([CH3,$(CH2CH3)])')
def test(x):
    mol=x['mol']
    mw=x['mol_weight']
    return mw<1000 and (mol.HasSubstructMatch(epoxide) or mol.HasSubstructMatch(aziridine))



[11:30:50] SMARTS Parse Error: syntax error while parsing: c1cn1([CH3,$(CH2CH3)_100])
[11:30:50] SMARTS Parse Error: Failed parsing SMARTS 'c1cn1([CH3,$(CH2CH3)_100])' for input: 'c1cn1([CH3,$(CH2CH3)])'
